## A demo to perform image differencing with TransFinder

## Install TransFinder through terminal
* Download the package: git clone https://github.com/LiuDezi/TransFinder.git
* Enter TransFinder dorectory: cd TransFinder
* Install: python -m pip install .

In [ ]:
# import necessary modules
from transfinder import imgdiff
from astropy.table import Table
import numpy as np
import os, sys, time

In [ ]:
# define the input
input_path = "/Users/dzliu/Workspace/TransFinder/tests/input"
output_path = "/Users/dzliu/Workspace/TransFinder/tests/output"
config_path = "/Users/dzliu/Workspace/TransFinder/config"

# input images
ref_image = "my_sc_tm33-07_r_20240128133113_022_sciimg.ref.fits"
sci_image = "my_sc_tm33-02_r_20240120141109_094_sciimg.fits"
ref_image_abs = os.path.join(input_path, ref_image)
sci_image_abs = os.path.join(input_path, sci_image)

# input swarp and sextractor configurations
sex_config_file = os.path.join(config_path, "default_config.sex")
sex_param_file = os.path.join(config_path, "default_param.sex")
swarp_config_file = os.path.join(config_path, "default_config.swarp")
swarp_exe = imgdiff.swarp_shell()
sex_exe = imgdiff.sextractor_shell()

survey_mode = "mephisto_pilot"

# output images
new_image = sci_image[:-4] + "new.fits"
diff_image = new_image[:-4] + "diff.fits"
new_image_abs = os.path.join(output_path, new_image)
diff_image_abs = os.path.join(output_path, diff_image)

In [ ]:
# build new image
t0 = time.time()
photcal_fig = os.path.join(output_path, "photcal.png")
imgdiff.build_newimg(sci_image_abs, new_image_abs, swarp_config_file, sex_config_file, sex_param_file,
                     swarp_exe = swarp_exe, sex_exe = sex_exe, survey_mode=survey_mode,
                     ref_image = ref_image_abs, ref_meta = None,
                     interp_badpixel_mode = "interp", interp_badpixel_grid = (100,100), interp_badpixel_flag = None, 
                     photcal_figure=photcal_fig,)
t1 = time.time()
dt = t1-t0
print(f"^_^ Total {dt} seconds used")

In [ ]:
# construct psf models for both reference and new images
t0 = time.time()
ref_meta = imgdiff.LoadMeta(ref_image_abs)
ref_mask = imgdiff.MaskStar(ref_meta,scale=1.5)
new_meta = imgdiff.LoadMeta(new_image_abs)
new_mask = imgdiff.MaskStar(new_meta,scale=1.5)

psf_size = np.max([ref_meta.psf_size, new_meta.psf_size])
psf_size = 31
print(f"    Input PSF size: (x_size, y_size)=({psf_size}, {psf_size})")

ref_psf_star_meta = imgdiff.PSFStar(ref_meta, psf_size=psf_size, nstar_max=500)
ref_psf_model = imgdiff.PSFModel(ref_psf_star_meta, ref_meta, info_frac=0.99, nbasis_max=3, poly_degree=3)
ref_psf_model.psf_model_diagnosis(ref_psf_star_meta, output_path=output_path, output_prefix=ref_image[:-5])

new_psf_star_meta = imgdiff.PSFStar(new_meta, psf_size=psf_size, nstar_max=500)
new_psf_model = imgdiff.PSFModel(new_psf_star_meta, new_meta, info_frac=0.99, nbasis_max=3, poly_degree=3)
new_psf_model.psf_model_diagnosis(new_psf_star_meta, output_path=output_path, output_prefix=new_image[:-5])
t1 = time.time()
dt = t1-t0
print(f"^_^ Total {dt} seconds used")

In [ ]:
# perform image differencing
t0 = time.time()
ng, dt, udt = [], [], []
for i in range(0, 50):
    ingrid = i + 1
    idt = []
    for j in range(20):
        t0 = time.time()
        diff_obj = imgdiff.DiffImg(degrid=(ingrid,ingrid), nthreads=-1)
        diff_matrix = diff_obj.diff(ref_meta, new_meta, ref_psf_model, new_psf_model,ref_mask=ref_mask.mask, new_mask=new_mask.mask)
        t1 = time.time()
        jdt = t1-t0
        idt.append(jdt)
    dt.append(np.mean(idt))
    udt.append(np.std(idt))
    ng.append(ingrid)
    print(f"^_^ N={ingrid}: {dt[-1]}+/-{udt[-1]} seconds on average used")

In [ ]:
import pylab as pl
pl.plot(ng, dt, "--", color="black", ms=3)
pl.xlabel("number of grids")
pl.ylabel("running time (seconds)")

In [ ]:
# perform source detection on the difference image
t0 = time.time()
det_obj = imgdiff.ExtractTrans(sex_config_file, sex_param_file, sex_exe=sex_exe,)
det_obj.extract_trans(diff_image_abs, diff_matrix, ref_meta, new_meta, cutout_write=True, cutout_path=output_path)
t1 = time.time()
dt = t1-t0
print(f"^_^ Total {dt} seconds used")